# 4-3: Check QARTOD test variables for tests executed

2025-06-13

The QARTOD `_qartod_executed` variables for some parameters have been missing the `tests_executed` attribute that lists as a string which tests were run and indicates which of the results in each string element correspond to specific test results.

Variables with the observed issue:
<ul>
    <li>GA01SUMO-RII11-02-PCO2WC051/pco2w_abc_imodem_instrument_recovered/pco2_seawater</li>
    <li>GS01SUMO-SBD11-08-NUTNRB000/nutnr_b_dcl_dark_full_instrument_recovered/nitrate_concentration</li>
</ul>

Slightly different observed abnormality, where the lookup tables exist and tests are in production (?), but no tests were executed.
<ul>
    <li>CP10CNSM-MFD37-03-CTDBPC000/ctdbp_cdef_dcl_instrument/pressure, temp, practical_salinity</li>
</ul>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import xarray as xr
import requests
import io
import ast
from glob import glob

In [25]:
# Import functions from ooinet and ooi_data_explorations libraries
from ooi_data_explorations.common import load_kdata, get_vocabulary, m2m_request, m2m_collect
from ooinet import M2M

In [3]:
# Import functions from project qc_completion module
from qartod_testing.qc_completion import load_gross_range_qartod_test_list, \
    load_climatology_qartod_test_list, make_test_parameter_dict, \
    check_tests_exe, make_results_table, add_test_exe, write_results

### GA01SUMO-RII11-02-PCO2WC051
Affected stream: pco2w_abc_imodem_instrument_recovered 

Affected variables: pco2_seawater

In [33]:
refdes = "GA01SUMO-RII11-02-PCO2WC051"
site, node, sensor = refdes.split("-", 2)
method = "recovered_host"
stream = "pco2w_abc_imodem_instrument_recovered"
deploy = 1
instclass = sensor[3:8]
data = load_kdata(site, node, sensor, method,
                      stream, ('*deployment%04d*%s*.nc' % (deploy, instclass))) # no regex
if data is None:
    m2m_result = m2m_request(site, node, sensor, method, stream)
    data = m2m_collect(m2m_result, tag=('.*deployment%04d.*%s.*.nc$' % (deploy, instclass))) # yes regex


Loading and Processing Data Files: 100%|██████████| 2/2 [00:00<00:00, 20.40it/s]

Merging the data files into a single dataset


In [34]:
data

<xarray.Dataset>
Dimensions:                                   (time: 1594, spectrum: 14)
Coordinates:
  * spectrum                                  (spectrum) int32 0 1 2 ... 12 13
  * time                                      (time) datetime64[ns] 2015-03-1...
Data variables: (12/26)
    pco2w_thermistor_temperature_qc_executed  (time) uint8 1 1 1 1 1 ... 1 1 1 1
    pco2_seawater_qartod_results              (time) float32 1.0 1.0 ... 255.0
    pco2_seawater_qc_executed                 (time) float32 29.0 29.0 ... 29.0
    absorbance_ratio_434_qc_executed          (time) float32 9.0 9.0 ... 9.0 9.0
    absorbance_blank_620                      (time) float64 2.412e+04 ... 2....
    record_type                               (time) uint8 4 4 4 4 4 ... 4 4 4 4
    ...                                        ...
    absorbance_ratio_620_qc_executed          (time) float32 9.0 9.0 ... 9.0 9.0
    record_time                               (time) uint32 3509301609 ... 35...
    absorbance_ratio_620_qc_results           (time) float32 9.0 9.0 ... 9.0 9.0
    thermistor_raw                            (time) int16 2013 2010 ... 2153
    pco2_seawater                             (time) float64 337.3 ... 363.5
    unique_id                                 (time) uint8 122 122 ... 122 122
Attributes: (12/69)
    node:                               RII11
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_host
    stream:                             pco2w_abc_imodem_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                -42.92283
    lon:                                -42.4393

Neither the `pco2_seawater_qartod_results` nor `pco2_seawater_qartod_executed` parameters have any attributes. This is what is making the test executed check fail. The last value in the QARTOD results is also not a valid QARTOD flag ("255"). The `qartod_executed` string for this data point is empty.

In [ ]:
data["pco2_seawater_qartod_results"]

In [ ]:
data["pco2_seawater_qartod_executed"]

## GS01SUMO-SBD11-08-NUTNRB000
Affected stream: nutnr_b_dcl_dark_full_instrument_recovered

Affected variables: nitrate_concentration

In [35]:
refdes = "GS01SUMO-SBD11-08-NUTNRB000"
site, node, sensor = refdes.split("-", 2)
method = "recovered_host"
stream = "nutnr_b_dcl_dark_full_instrument_recovered"
deploy = 1
instclass = sensor[3:8]
data = load_kdata(site, node, sensor, method,
                      stream, ('*deployment%04d*%s*.nc' % (deploy, instclass))) # no regex
if data is None:
    m2m_result = m2m_request(site, node, sensor, method, stream)
    data = m2m_collect(m2m_result, tag=('.*deployment%04d.*%s.*.nc$' % (deploy, instclass))) # yes regex


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00,  6.13it/s]

Merging the data files into a single dataset


In [36]:
data

<xarray.Dataset>
Dimensions:                                (time: 3696, string5: 5,
                                            wavelength: 256, string3: 3,
                                            string4: 4)
Coordinates:
  * wavelength                             (wavelength) int32 0 1 2 ... 254 255
  * time                                   (time) datetime64[ns] 2015-02-18T2...
Dimensions without coordinates: string5, string3, string4
Data variables: (12/30)
    ref_channel_variance                   (time) float32 2.7 2.29 ... 2.31 2.85
    deployment                             (time) int32 1 1 1 1 1 ... 1 1 1 1 1
    dcl_controller_timestamp               (time, string5) |S1 b'e' ... b'y'
    voltage_analog                         (time) float32 4.98 5.03 ... 4.99
    nitrate_concentration_qartod_executed  (time) object '33' '33' ... '33' '33'
    dark_frame_spectral_channels           (time, wavelength) uint16 765 ... 778
    ...                                     ...
    nitrate_concentration_qc_executed      (time) uint8 29 29 29 29 ... 29 29 29
    date_of_sample                         (time) int32 -9999 -9999 ... -9999
    humidity                               (time) float32 6.46 6.53 ... 7.56
    aux_fitting_1                          (time) float32 0.0 0.0 ... 0.0 0.0
    aux_fitting_2                          (time) float32 0.0 0.0 ... 0.0 0.0
    voltage_lamp                           (time) float32 11.96 11.96 ... 11.98
Attributes: (12/69)
    node:                               SBD11
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_host
    stream:                             nutnr_b_dcl_dark_full_instrument_reco...
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                -54.4082
    lon:                                -89.3576

In [ ]:
data["pco2_seawater_qartod_results"]

In [ ]:
data["pco2_seawater_qartod_executed"]

## CP10CNSM-MFD37-03-CTDBPC000
Affected stream: ctdbp_cdef_dcl_instrument

Affected variables: pressure, temp, practical_salinity

After using get_vocabulary and changing to BASH not RegEx wildcards, data loaded correctly, and the qartod results were present. How did that happen?

In [37]:
refdes = "CP10CNSM-MFD37-03-CTDBPC000"
site, node, sensor = refdes.split("-", 2)
method = "telemetered"
stream = "ctdbp_cdef_dcl_instrument"
deploy = 1
instclass = sensor[3:8]
get_vocabulary(site, node, sensor)
data = load_kdata(site, node, sensor, method,
                      stream, ('*deployment%04d*%s*.nc' % (deploy, instclass)))


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00,  5.49it/s]

Merging the data files into a single dataset


In [38]:
data

<xarray.Dataset>
Dimensions:                                       (time: 2470, string5: 5)
Coordinates:
  * time                                          (time) datetime64[ns] 2024-...
Dimensions without coordinates: string5
Data variables: (12/16)
    sea_water_pressure                            (time) float32 29.67 ... 30.52
    dcl_controller_timestamp                      (time, string5) |S1 b'e' .....
    sea_water_temperature_qartod_results          (time) uint8 1 1 1 1 ... 1 1 1
    internal_timestamp                            (time) datetime64[ns] 2024-...
    sea_water_pressure_qartod_executed            (time) object '1' '1' ... '1'
    date_time_string                              (time, string5) |S1 b'e' .....
    ...                                            ...
    sea_water_temperature                         (time) float32 9.209 ... 15.06
    sea_water_density                             (time) float64 1.025e+03 .....
    depth                                         (time) float64 29.45 ... 30.29
    sea_water_pressure_qartod_results             (time) uint8 1 1 1 1 ... 1 1 1
    sea_water_electrical_conductivity             (time) float32 3.47 ... 3.893
    sea_water_practical_salinity_qartod_executed  (time) object '1' '1' ... '1'
Attributes: (12/69)
    node:                               MFD37
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             ctdbp_cdef_dcl_instrument
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                35.94988
    lon:                                -75.11943

## GS03FLMB-RIS01-05-FLORTD000

Affected stream: flort_sample

In [22]:
refdes = "GS03FLMB-RIS01-05-FLORTD000"
site, node, sensor = refdes.split("-", 2)
method = "telemetered"
stream = "flort_sample"
deploy = 1
instclass = sensor[3:8]
get_vocabulary(site, node, sensor)
data = load_kdata(site, node, sensor, method,
                      stream, ('*deployment%04d*%s*.nc' % (deploy, instclass)))
# data = load_kdata(site, node, sensor, method,
#                       stream, ('*deployment%04d*.nc' % (deploy)))


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00, 24.78it/s]

Merging the data files into a single dataset


In [23]:
data

<xarray.Dataset>
Dimensions:                                          (time: 562)
Coordinates:
  * time                                             (time) datetime64[ns] 20...
Data variables: (12/31)
    fluorometric_chlorophyll_a_qc_results            (time) uint8 13 13 ... 5 5
    fluorometric_cdom                                (time) float64 0.9999 .....
    fluorometric_cdom_qc_executed                    (time) uint8 29 29 ... 29
    fluorometric_cdom_qartod_results                 (time) uint8 1 1 1 ... 1 1
    fluorometric_chlorophyll_a_qartod_results        (time) uint8 1 1 1 ... 1 1
    fluorometric_cdom_qartod_executed                (time) object '11' ... '11'
    ...                                               ...
    fluorometric_chlorophyll_a                       (time) float64 0.5808 .....
    measurement_wavelength_cdom                      (time) uint16 460 ... 460
    suspect_timestamp                                (time) int8 -99 -99 ... -99
    fluorometric_chlorophyll_a_qartod_executed       (time) object '11' ... '11'
    fluorometric_cdom_qc_results                     (time) uint8 13 13 ... 13
    sea_water_temperature                            (time) float64 7.751 ......
Attributes: (12/69)
    node:                               RIS01
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             flort_sample
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                -54.0803
    lon:                                -88.9346

In [29]:
m2m_results = m2m_request(site, node, sensor,
                          method, stream)

Requesting:
	refdes: GS03FLMB-RIS01-05-FLORTD000
	method: telemetered
	stream: flort_sample
	from None to None
Waiting for OOINet to process and prepare data request, this may take up to 20 minutes.

Waiting: 100%|██████████| 400/400 [00:30<00:00, 13.26it/s][A


In [30]:
m2m_results

{'requestUUID': '68d72dd3-672e-4c07-a198-ce732b228899',
 'outputURL': 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/kylene.cooley@whoi.edu/20250806T190532859Z-GS03FLMB-RIS01-05-FLORTD000-telemetered-flort_sample/catalog.html',
 'allURLs': ['https://opendap.oceanobservatories.org/thredds/catalog/ooi/kylene.cooley@whoi.edu/20250806T190532859Z-GS03FLMB-RIS01-05-FLORTD000-telemetered-flort_sample/catalog.html',
  'https://downloads.oceanobservatories.org/async_results/kylene.cooley@whoi.edu/20250806T190532859Z-GS03FLMB-RIS01-05-FLORTD000-telemetered-flort_sample'],
 'sizeCalculation': 891000,
 'timeCalculation': 60,
 'numberOfSubJobs': 179}

In [31]:
m2m_data = m2m_collect(m2m_results)

Merging the data files into a single dataset


In [32]:
m2m_data

<xarray.Dataset>
Dimensions:                                          (time: 937)
Coordinates:
  * time                                             (time) datetime64[ns] 20...
Data variables: (12/31)
    fluorometric_chlorophyll_a_qc_results            (time) float32 13.0 ... ...
    fluorometric_cdom                                (time) float64 1.246 ......
    fluorometric_cdom_qc_executed                    (time) float32 29.0 ... ...
    fluorometric_cdom_qartod_results                 (time) float32 1.0 ... 1.0
    fluorometric_chlorophyll_a_qartod_results        (time) float32 1.0 ... 1.0
    fluorometric_cdom_qartod_executed                (time) object '11' ... '11'
    ...                                               ...
    fluorometric_chlorophyll_a                       (time) float64 0.5808 .....
    measurement_wavelength_cdom                      (time) float32 460.0 ......
    suspect_timestamp                                (time) float32 -99.0 ......
    fluorometric_chlorophyll_a_qartod_executed       (time) object '11' ... '11'
    fluorometric_cdom_qc_results                     (time) float32 13.0 ... ...
    sea_water_temperature                            (time) float64 7.751 ......
Attributes: (12/69)
    node:                               RIS01
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             flort_sample
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                -54.0824
    lon:                                -88.85905

## GS03FLMA-RIS01-05-FLORTD000 and 03-DOSTAD000

Checking examples of sensors with data containing the correct QARTOD attributes.

In [39]:
refdes = "GS03FLMA-RIS01-05-FLORTD000"
site, node, sensor = refdes.split("-", 2)
method = "recovered_host"
stream = "flort_sample"
deploy = 1
instclass = sensor[3:8]
get_vocabulary(site, node, sensor)
data = load_kdata(site, node, sensor, method,
                      stream, ('*deployment%04d*%s*.nc' % (deploy, instclass)))
data


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00,  4.92it/s]

Merging the data files into a single dataset


<xarray.Dataset>
Dimensions:                                          (time: 24691)
Coordinates:
  * time                                             (time) datetime64[ns] 20...
Data variables: (12/29)
    fluorometric_chlorophyll_a_qc_results            (time) uint8 29 29 ... 29
    fluorometric_cdom                                (time) float64 0.6349 .....
    fluorometric_cdom_qc_executed                    (time) uint8 29 29 ... 29
    fluorometric_cdom_qartod_results                 (time) uint8 1 1 1 ... 1 1
    fluorometric_chlorophyll_a_qartod_results        (time) uint8 1 1 1 ... 1 1
    fluorometric_cdom_qartod_executed                (time) object '11' ... '11'
    ...                                               ...
    total_volume_scattering_coefficient              (time) float64 0.0001076...
    fluorometric_chlorophyll_a                       (time) float64 0.8052 .....
    measurement_wavelength_cdom                      (time) uint16 460 ... 460
    suspect_timestamp                                (time) int8 -99 -99 ... -99
    fluorometric_chlorophyll_a_qartod_executed       (time) object '11' ... '11'
    fluorometric_cdom_qc_results                     (time) uint8 29 29 ... 29
Attributes: (12/69)
    node:                               RIS01
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_host
    stream:                             flort_sample
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                -54.10517
    lon:                                -89.65361

In [40]:
refdes = "GS03FLMA-RIS01-03-DOSTAD000"
site, node, sensor = refdes.split("-", 2)
method = "telemetered"
stream = "dosta_abcdjm_sio_instrument"
deploy = 1
instclass = sensor[3:8]
get_vocabulary(site, node, sensor)
data = load_kdata(site, node, sensor, method,
                      stream, ('*deployment%04d*%s*.nc' % (deploy, instclass)))
data


Loading and Processing Data Files: 100%|██████████| 1/1 [00:00<00:00, 17.78it/s]

Merging the data files into a single dataset


<xarray.Dataset>
Dimensions:                                         (time: 563)
Coordinates:
  * time                                            (time) datetime64[ns] 201...
Data variables: (12/35)
    raw_temperature                                 (time) float32 468.4 ... ...
    dosta_abcdjm_cspp_tc_oxygen_qc_executed         (time) uint8 13 13 ... 13 13
    estimated_oxygen_concentration_qc_executed      (time) uint8 9 9 9 ... 9 9 9
    red_phase                                       (time) float32 8.902 ... ...
    dosta_abcdjm_cspp_tc_oxygen_qc_results          (time) uint8 13 13 ... 13 13
    estimated_oxygen_concentration_qartod_executed  (time) object '1' ... '1'
    ...                                              ...
    red_amplitude                                   (time) float32 892.8 ... ...
    sio_controller_timestamp                        (time) uint32 1424908801 ...
    depth                                           (time) float64 0.01268 .....
    blue_amplitude                                  (time) float32 888.2 ... ...
    estimated_oxygen_saturation_qc_executed         (time) uint8 1 1 1 ... 1 1 1
    sea_water_temperature                           (time) float64 7.198 ... ...
Attributes: (12/69)
    node:                               RIS01
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  telemetered
    stream:                             dosta_abcdjm_sio_instrument
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                -54.10517
    lon:                                -89.65361